In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import nilmtk

import os
import time
import datetime
#from tqdm import tqdm_notebook

import numpy as np
import pandas
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns

# Tensorflow >= 2.00 & Keras-gpu
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Activation, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Conv1D, LSTM, Bidirectional, Reshape
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import cifar10
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("keras version %s (should be at least 0.12.1)" % keras.__version__)

import tensorflow as tf
print("Tensorflow version is ",tf.__version__)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16772686014735439425
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12479430887944983480
physical_device_desc: "device: XLA_CPU device"
]
keras version 2.2.4-tf (should be at least 0.12.1)
Tensorflow version is  2.1.0


In [25]:
#df = pd.read_csv("Preprocessed_reviews.csv")
df = pd.read_pickle("Preprocessed_reviews.pkl")

In [26]:
df.head()

,Restaurant Name,Review ID,Date,Content,Rating,Review_filtered,Review_split,Review_Stem,Review-stopword
0,Madhu's,739447935,January 2020,The food was absolutely wonderful from prepara...,5.0,the food was absolutely wonderful from prepara...,"[the, food, was, absolutely, wonderful, from, ...","[the, food, be, absolutely, wonderful, from, p...","[food, absolutely, wonderful, preparation, pre..."
1,Madhu's,739773109,January 2020,The food was amazing. Our waiter Franco was ex...,5.0,the food was amazing our waiter franco was exc...,"[the, food, was, amazing, our, waiter, franco,...","[the, food, be, amaze, our, waiter, franco, be...","[food, amaze, waiter, franco, exceptional, att..."
2,Madhu's,738529074,January 2020,It was an awesome experience dining at the res...,5.0,it was an awesome experience dining at the res...,"[it, was, an, awesome, experience, dining, at,...","[it, be, an, awesome, experience, dining, at, ...","[awesome, experience, dining, restaurant, thur..."
3,Madhu's,739981237,January 2020,"Excellent service, excellent food and great am...",5.0,excellent service excellent food and great amb...,"[excellent, service, excellent, food, and, gre...","[excellent, service, excellent, food, and, gre...","[excellent, service, excellent, food, great, a..."
4,Madhu's,740251404,January 2020,Food was good cannot say too good but staff an...,4.0,food was good cannot say too good but staff an...,"[food, was, good, cannot, say, too, good, but,...","[food, be, good, cannot, say, too, good, but, ...","[food, good, say, good, staff, service, excell..."


In [27]:
df = df.sample(frac=1).reset_index(drop=True)

In [28]:
df.shape

(488413, 9)

In [29]:
df['length'] = df['Review_split'].map(lambda x: len(x))

In [35]:
df['length'].describe()

count    488413.000000
mean         57.963220
std          49.571208
min           0.000000
25%          26.000000
50%          44.000000
75%          74.000000
max        1158.000000
Name: length, dtype: float64

In [36]:
df = df.loc[:39999,['Review_filtered', 'Rating']]

In [49]:
max_features = 24000
max_words = 80
num_classes = 5

In [50]:
X_train = df.loc[:,['Review_filtered']]
Y_train = df.loc[:,['Rating']]
X_train = X_train['Review_filtered']
Y_train = Y_train['Rating']
X_train = X_train.astype(str)

In [51]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train_TK = tokenizer.texts_to_sequences(X_train)

In [52]:
X_train_TKP = pad_sequences(X_train_TK, maxlen=max_words)

In [53]:
from tensorflow.keras.utils import to_categorical

Y_train_OHC = to_categorical(Y_train - 1, num_classes)


In [54]:
print(X_train_TKP.shape)
print(Y_train_OHC.shape)


(40000, 80)
(40000, 5)


In [59]:
model3_LSTM = Sequential()
model3_LSTM.add(Embedding(max_features, 256, input_length=max_words, mask_zero=True, name="Embed"))
model3_LSTM.add(Bidirectional(LSTM(256, dropout=.5, recurrent_dropout=.5, return_sequences=True, name="LSTM1")))
model3_LSTM.add(Bidirectional(LSTM(64, dropout=.5, return_sequences=False, name="LSTM2")))
model3_LSTM.add(Dense(num_classes, activation='softmax', name="FC1"))
model3_LSTM.compile(loss='categorical_crossentropy',
                    optimizer=Adam(),
                    metrics=['accuracy'])
model3_LSTM.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embed (Embedding)            (None, 80, 256)           6144000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 80, 512)           1050624   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               295424    
_________________________________________________________________
FC1 (Dense)                  (None, 5)                 645       
Total params: 7,490,693
Trainable params: 7,490,693
Non-trainable params: 0
_________________________________________________________________


In [60]:
history3=model3_LSTM.fit(X_train_TKP, Y_train_OHC, validation_split=0.2, epochs=30, batch_size=256, verbose=1)

Train on 32000 samples, validate on 8000 samples
Epoch 1/30
32000/32000 [==============================] - 590s 18ms/sample - loss: 1.0944 - accuracy: 0.5314 - val_loss: 0.9791 - val_accuracy: 0.5794
Epoch 2/30
32000/32000 [==============================] - 596s 19ms/sample - loss: 0.8731 - accuracy: 0.6237 - val_loss: 0.9234 - val_accuracy: 0.6035
Epoch 3/30
32000/32000 [==============================] - 597s 19ms/sample - loss: 0.7739 - accuracy: 0.6760 - val_loss: 0.9521 - val_accuracy: 0.6009
Epoch 4/30
14848/32000 [============>.................] - ETA: 4:55 - loss: 0.6754 - accuracy: 0.7265

KeyboardInterrupt: 